In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain_core.documents import Document
import fitz
import concurrent.futures
from langchain_aws import BedrockEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field

import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool



In [2]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b
    
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    # HTTP GET 요청 보내기
    response = requests.get(news_url)

    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]


In [3]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")

In [4]:
tools

[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x7f499610e2a0>),
 StructuredTool(name='add_function', description='Adds two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.add_function'>, func=<function add_function at 0x7f499610e200>),
 StructuredTool(name='naver_news_crawl', description='Crawls a 네이버 (naver.com) news article and returns the body content.', args_schema=<class 'langchain_core.utils.pydantic.naver_news_crawl'>, func=<function naver_news_crawl at 0x7f499610e340>)]

In [5]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)
chat = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)

In [6]:
#tool과 연동
llm_with_tools = chat.bind_tools(tools)


In [8]:
#어떤 툴을 사용해야 하지는 llm이 json형태로 알려줌
llm_with_tools.invoke("What is the length of the word 'AnyBank'?").tool_calls


[{'name': 'get_word_length',
  'args': {'word': 'AnyBank'},
  'id': 'tooluse_eGkKhN9KSvC2hr-wl8oSnQ',
  'type': 'tool_call'}]

In [17]:
tools

[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x7f499610e2a0>),
 StructuredTool(name='add_function', description='Adds two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.add_function'>, func=<function add_function at 0x7f499610e200>),
 StructuredTool(name='naver_news_crawl', description='Crawls a 네이버 (naver.com) news article and returns the body content.', args_schema=<class 'langchain_core.utils.pydantic.naver_news_crawl'>, func=<function naver_news_crawl at 0x7f499610e340>)]

In [18]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser


In [19]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'AnyBank'?")


In [20]:
tool_call_results

[{'args': {'word': 'AnyBank'}, 'type': 'get_word_length'}]

In [21]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': 'AnyBank'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'AnyBank'}


In [22]:
execute_tool_calls(tool_call_results)


[실행도구] get_word_length [Argument] {'word': 'AnyBank'}
[실행결과] 7


In [26]:

from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [27]:
chain.invoke("What is the length of the word 'AnyBank'?")


[실행도구] get_word_length [Argument] {'word': 'AnyBank'}
[실행결과] 7


In [28]:
chain.invoke("114.5 + 121.2")

[실행도구] add_function [Argument] {'a': 114.5, 'b': 121.2}
[실행결과] 235.7


In [29]:
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)


[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}
[실행결과] [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년

In [34]:
#AgentExecutor 및 create_tool_calling_agent 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [35]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [36]:
tools = [get_word_length, add_function, naver_news_crawl]


In [37]:
agent = create_tool_calling_agent(chat, tools, prompt)


In [38]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)


In [39]:
agent_executor.invoke({"input": "How many letters in the word `AnyBank`?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'AnyBank'}`
responded: [{'type': 'text', 'text': 'To get the length of the word "AnyBank", we can use the `get_word_length` tool:', 'index': 0}, {'type': 'tool_use', 'name': 'get_word_length', 'id': 'tooluse_4COGpsckREuxz3LrvQ7oug', 'index': 1, 'input': '{"word": "AnyBank"}'}]

[{'type': 'text', 'text': '\n\nThe word "AnyBank" has 7 letters.', 'index': 0}]

> Finished chain.


{'input': 'How many letters in the word `AnyBank`?',
 'output': [{'type': 'text',
   'text': '\n\nThe word "AnyBank" has 7 letters.',
   'index': 0}]}

In [41]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`
responded: [{'type': 'tool_use', 'name': 'naver_news_crawl', 'id': 'tooluse_izAuIZqiTtSgUUDDiSXfrg', 'index': 0, 'input': '{"news_url": "https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"}'}]

[미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망